In [2]:
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, IntVector, Formula
pandas2ri.activate()

%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

base = importr('base')
stats = importr('stats')
ergm = importr('ergm')
network = importr('network', on_conflict="warn",
                  robject_translations={
                      'as.tibble.network': 'as_tibble_network',
                      'as_tibble.network': 'as_tibble_network'
                  }
         )

/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '
/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/packages_utils.py:127: UserWarning: Conflict when converting R symbols in the package "network" to Python symbols: 
-as_tibble_network -> as_tibble.network, as.tibble.network
  warn(msg)


## Load data

In [3]:
filename = '../data/mit.rds'
readRDS = robjects.r['readRDS']
res = readRDS(filename)
df = np.array(res).astype(int)

In [4]:
df.shape

(232, 96, 96)

In [5]:
df[0,:10,:10]

array([[0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0]])

In [14]:
num_nodes = df[0].shape[0]
# n = 20
T = len(df)
# num_changepts = 5

p_pos = 2 # edge + mutual
p_neg = 2 # edge + mutual
p = p_pos + p_neg

np.random.seed(42)
theta = np.random.normal(0, 0.5, size= (T , p)) 
z = np.copy(theta) 
u = np.zeros((T, p)) 


form_terms = ['edges', 'mutual']
diss_terms = ['edges', 'mutual']

In [15]:
init = {
    'theta': np.copy(theta),
    'z': z,
    'u': u
}

In [16]:
from mple_learn import stergmGraph

In [17]:
sim1 = stergmGraph(
    X = df,
    form_terms=form_terms,
    diss_terms=diss_terms,
    lam=[1],
    max_steps=50,
    newton_max_steps=100,
    admm_alpha=1,
    converge_tol=1e-5,
    verbose=1
)

In [18]:
theta_res, z, u, c = sim1.mple(initial_values=init, solver='newton', tau_inc=2, m=10)

[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 3.978688955307007.
[INFO] Updating z...
[INFO] Updating u...
-------------------------------------------------
[INFO] decreasing alpha to 0.5
[INFO] max mu :  0.9966788240436449
[INFO] dual_resnorm: 5.926684
[INFO] primal_resnorm: 0.477283
[INFO] convergence: 5.926684
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 2.395137071609497.
[INFO] Updating z...
[INFO] Updating u...
[INFO] max mu :  0.9996373358947608
[INFO] dual_resnorm: 2.601189
[INFO] primal_resnorm: 0.697271
[INFO] convergence: 2.601189
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 2.02933406829834.
[INFO] Updating z...
[INFO] Updating u...
[INFO] max mu :  0.9998738016033651
[INFO] dual_resnorm: 1.154761
[INFO] primal_resnorm: 0.122335
[INFO] convergence: 1.154761
-------------------------------------

[INFO] Newton converged in: 1.3402328491210938.
[INFO] Updating z...
[INFO] Updating u...
[INFO] max mu :  0.999999157655817
[INFO] dual_resnorm: 0.091532
[INFO] primal_resnorm: 0.041028
[INFO] convergence: 0.091532
-------------------------------------------------
[INFO] ADMM step #27
[INFO] Updating theta...
[INFO] Newton converged in: 1.3410019874572754.
[INFO] Updating z...
[INFO] Updating u...
[INFO] max mu :  0.9999992429797708
[INFO] dual_resnorm: 0.086460
[INFO] primal_resnorm: 0.042688
[INFO] convergence: 0.086460
-------------------------------------------------
[INFO] ADMM step #28
[INFO] Updating theta...
[INFO] Newton converged in: 1.3321051597595215.
[INFO] Updating z...
[INFO] Updating u...
[INFO] max mu :  0.9999993187437093
[INFO] dual_resnorm: 0.084350
[INFO] primal_resnorm: 0.051717
[INFO] convergence: 0.084350
-------------------------------------------------
[INFO] ADMM step #29
[INFO] Updating theta...
[INFO] Newton converged in: 1.3631529808044434.
[INFO] Updatin

In [19]:
from utils import estimate_changepts

In [21]:
theta_res.shape

(232, 4)

In [60]:
mask = np.linalg.norm(np.diff(init['theta'], axis=0), ord=2, axis=1) > 4

In [46]:
from datetime import date, timedelta
base = date(2004, 9, 15)
date_list = [base + timedelta(days=x) for x in range(231)]

In [47]:
len(mask)

231

In [61]:
np.array(date_list)[mask]

array([datetime.date(2004, 9, 15), datetime.date(2004, 12, 30),
       datetime.date(2005, 1, 2), datetime.date(2005, 1, 3)], dtype=object)

In [55]:
date_list

[datetime.date(2004, 9, 15),
 datetime.date(2004, 9, 16),
 datetime.date(2004, 9, 17),
 datetime.date(2004, 9, 18),
 datetime.date(2004, 9, 19),
 datetime.date(2004, 9, 20),
 datetime.date(2004, 9, 21),
 datetime.date(2004, 9, 22),
 datetime.date(2004, 9, 23),
 datetime.date(2004, 9, 24),
 datetime.date(2004, 9, 25),
 datetime.date(2004, 9, 26),
 datetime.date(2004, 9, 27),
 datetime.date(2004, 9, 28),
 datetime.date(2004, 9, 29),
 datetime.date(2004, 9, 30),
 datetime.date(2004, 10, 1),
 datetime.date(2004, 10, 2),
 datetime.date(2004, 10, 3),
 datetime.date(2004, 10, 4),
 datetime.date(2004, 10, 5),
 datetime.date(2004, 10, 6),
 datetime.date(2004, 10, 7),
 datetime.date(2004, 10, 8),
 datetime.date(2004, 10, 9),
 datetime.date(2004, 10, 10),
 datetime.date(2004, 10, 11),
 datetime.date(2004, 10, 12),
 datetime.date(2004, 10, 13),
 datetime.date(2004, 10, 14),
 datetime.date(2004, 10, 15),
 datetime.date(2004, 10, 16),
 datetime.date(2004, 10, 17),
 datetime.date(2004, 10, 18),
 date